## Problem 1 — Pick-and-Place Robot (MDP design)
Goal: Formulate the pick-and-place task as an MDP and justify each component.

1. States (S):
The agent's state should capture everything needed to make decisions. A practical choice is a vector containing: joint angles (q), joint velocities (q̇), end-effector pose (position and orientation), gripper status (open/closed), and the relative positions of the object and the placement target.
2. Actions (A):
Use continuous actions for low-level control to achieve smooth motion. Two common choices: direct motor torques (τ) for full control; or target joint velocities or small position deltas.
3. Transition dynamics (P):
Transitions follow robot dynamics: s_{t+1} = f(s_t, a_t) + noise. Simulators give deterministic physics; real systems add small stochasticity.
4. Reward function (R):
The reward balances task completion, speed, smoothness, and safety. A compact and practical reward is:
r_t = I_success * R_success - c_time - c_effort * ||a_t||^2 - c_smooth * ||a_t - a_{t-1}||^2 - I_collision * R_collision + c_progress * (previous_distance - current_distance).
5. Discount factor (γ):
Pick γ close to 1 (e.g., 0.95–0.99) since the task horizon is moderate and we want to value future success.
6. Algorithm suggestions and practical tips:
Best algorithms for continuous control: Soft Actor-Critic (SAC) or TD3 for off-policy learning, and PPO if you prefer an on-policy method. Use simulation, normalize inputs, clip actions, and consider pretraining with demonstrations.

Note: I have chosen continuous joint-velocity style actions for safety and added both time and smoothness penalties because the assignment emphasized fast and smooth movements.

## Problem 2: 2x2 Gridworld 
Consider a 2x2 gridworld with four states (s1, s2, s3, s4). The agent can move up, down, left, or right. If an action leads into a wall, the agent stays in the same state. Rewards are state-based: R(s1)=5, R(s2)=10, R(s3)=1, and R(s4)=2. The initial policy always selects 'up'.  Then perform two iterations of value iteration.

## Iteration 1
## Step 1:  Initialize the value function V(s)=0 for all states.
s1	s2	s3	s4
0	0	0	0

## Step 2:  Update the value function using Bellman optimality updates:
V(s) ← max_a [ R(s) + γ * V(s′) ]
Assume γ = 0.9.

## Step 3:  Since initial V(s)=0, the first update gives:
s1	s2	s3	s4
5	10	1	2

## Iteration 2
Using the updated values from Iteration 1,  to perform another Bellman update. Now, each state value is influenced not just by its immediate reward but also by the discounted value of the best reachable next state.
The updated values after Iteration 2 are approximately:
s1	s2	s3	s4
14	19	10	11

## Conclusion
In summary, Problem 1 demonstrated how reinforcement learning can be applied to real-world robotic tasks by designing an appropriate MDP with states, actions, and rewards. Problem 2 highlighted how value iteration works step by step, gradually improving the state-value function estimates until convergence. These problems illustrate the importance of reward design and iterative updates in teaching agents to learn effective policies.

## Problem 3 — 5×5 Gridworld

1. Environment Setup
Implement a 5×5 Gridworld where the agent can move in four directions:
•	Actions: R (right), D (down), L (left), U (up)
•	Goal: (4,4) with reward +10
•	Grey states: (2,2), (3,0), (0,4) with penalty -5
•	All other states: -1 reward per step
•	Discount factor: γ = 0.9
Movement outside the grid leaves the agent in the same state.


In [2]:
%pip install numpy

   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.8 MB 7.2 MB/s eta 0:00:02
   -------- ------------------------------- 2.6/12.8 MB 7.6 MB/s eta 0:00:02
   ------------ --------------------------- 3.9/12.8 MB 7.3 MB/s eta 0:00:02
   ------------------ --------------------- 5.8/12.8 MB 7.4 MB/s eta 0:00:01
   ---------------------- ----------------- 7.3/12.8 MB 7.5 MB/s eta 0:00:01
   --------------------------- ------------ 8.9/12.8 MB 7.4 MB/s eta 0:00:01
   ------------------------------- -------- 10.2/12.8 MB 7.1 MB/s eta 0:00:01
   ---------------------------------- ----- 11.0/12.8 MB 6.9 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.8 MB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 6.6 MB/s  0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# value_iteration_gridworld.py
import numpy as np
import time

ROWS, COLS = 5, 5
ACTIONS = ['R','D','L','U']
ACTION_TO_DELTA = {'R': (0,1), 'D': (1,0), 'L': (0,-1), 'U': (-1,0)}
GAMMA = 0.9

def in_bounds(r,c):
    return 0 <= r < ROWS and 0 <= c < COLS

def step(rc, action):
    r,c = rc
    dr,dc = ACTION_TO_DELTA[action]
    nr, nc = r + dr, c + dc
    if in_bounds(nr, nc):
        return (nr, nc)
    else:
        return (r, c)

# Reward map per problem statement
REWARDS = np.full((ROWS, COLS), -1.0)
for (r,c) in [(2,2), (3,0), (0,4)]:  # grey states
    REWARDS[r,c] = -5.0
GOAL = (4,4)
REWARDS[GOAL] = 10.0

def value_iteration(rewards, gamma=GAMMA, theta=1e-5):
    V = np.zeros_like(rewards)
    iters = 0
    while True:
        delta = 0.0
        V_new = V.copy()
        for r in range(ROWS):
            for c in range(COLS):
                if (r,c) == GOAL:
                    V_new[r,c] = rewards[r,c]
                    continue
                q_vals = []
                for a in ACTIONS:
                    nr, nc = step((r,c), a)
                    q_vals.append(rewards[r,c] + gamma * V[nr, nc])
                V_new[r,c] = max(q_vals)
                delta = max(delta, abs(V_new[r,c] - V[r,c]))
        V = V_new
        iters += 1
        if delta < theta:
            break
    # extract greedy policy
    policy = np.full((ROWS, COLS), '', dtype=object)
    for r in range(ROWS):
        for c in range(COLS):
            if (r,c) == GOAL:
                policy[r,c] = 'G'
            else:
                best_a = None
                best_q = -1e9
                for a in ACTIONS:
                    nr, nc = step((r,c), a)
                    q = rewards[r,c] + gamma * V[nr,nc]
                    if q > best_q:
                        best_q = q; best_a = a
                policy[r,c] = best_a
    return V, policy, iters

def inplace_value_iteration(rewards, gamma=GAMMA, theta=1e-5):
    V = np.zeros_like(rewards)
    iters = 0
    while True:
        delta = 0.0
        for r in range(ROWS):
            for c in range(COLS):
                if (r,c) == GOAL:
                    old = V[r,c]; V[r,c] = rewards[r,c]; delta = max(delta, abs(V[r,c]-old)); continue
                old = V[r,c]
                q_vals = []
                for a in ACTIONS:
                    nr, nc = step((r,c), a)
                    q_vals.append(rewards[r,c] + gamma * V[nr, nc])  # uses updated V if neighbor already updated
                V[r,c] = max(q_vals)
                delta = max(delta, abs(V[r,c] - old))
        iters += 1
        if delta < theta:
            break
    # extract policy same way
    policy = np.full((ROWS, COLS), '', dtype=object)
    for r in range(ROWS):
        for c in range(COLS):
            if (r,c) == GOAL:
                policy[r,c] = 'G'
            else:
                best_a, best_q = None, -1e9
                for a in ACTIONS:
                    nr, nc = step((r,c), a)
                    q = rewards[r,c] + gamma * V[nr,nc]
                    if q > best_q: best_q = q; best_a = a
                policy[r,c] = best_a
    return V, policy, iters
if __name__ == "__main__":
    start = time.time()
    V1, policy1, iters1 = value_iteration(REWARDS)
    end = time.time()
    print("Standard Value Iteration:")
    print("Value Function:\n", V1)
    print("Policy:\n", policy1)
    print(f"Iterations: {iters1}, Time: {end-start:.6f} seconds\n")

    start = time.time()
    V2, policy2, iters2 = inplace_value_iteration(REWARDS)
    end = time.time()
    print("In-Place Value Iteration:")
    print("Value Function:\n", V2)
    print("Policy:\n", policy2)
    print(f"Iterations: {iters2}, Time: {end-start:.6f} seconds")

Standard Value Iteration:
Value Function:
 [[-1.3906558 -0.434062   0.62882    1.8098    -0.878    ]
 [-0.434062   0.62882    1.8098     3.122      4.58     ]
 [ 0.62882    1.8098    -0.878      4.58       6.2      ]
 [-2.1902     3.122      4.58       6.2        8.       ]
 [ 3.122      4.58       6.2        8.        10.       ]]
Policy:
 [['R' 'R' 'R' 'D' 'D']
 ['R' 'R' 'R' 'R' 'D']
 ['R' 'D' 'R' 'R' 'D']
 ['R' 'R' 'R' 'R' 'D']
 ['R' 'R' 'R' 'R' 'G']]
Iterations: 10, Time: 0.001759 seconds

In-Place Value Iteration:
Value Function:
 [[-1.3906558 -0.434062   0.62882    1.8098    -0.878    ]
 [-0.434062   0.62882    1.8098     3.122      4.58     ]
 [ 0.62882    1.8098    -0.878      4.58       6.2      ]
 [-2.1902     3.122      4.58       6.2        8.       ]
 [ 3.122      4.58       6.2        8.        10.       ]]
Policy:
 [['R' 'R' 'R' 'D' 'D']
 ['R' 'R' 'R' 'R' 'D']
 ['R' 'D' 'R' 'R' 'D']
 ['R' 'R' 'R' 'R' 'D']
 ['R' 'R' 'R' 'R' 'G']]
Iterations: 10, Time: 0.001624 seconds


3. Results

3.1 Convergence

| Method | Iterations | Time (seconds) |
|--------|------------|----------------|
| Standard Value Iteration | 10 | 0.0075 |
| In-place Value Iteration | 10 | 0.0040 |

Observation: In-place iteration ran slightly faster in this small grid, though both converged in the same number of iterations.

3.2 Optimal Value Function V*

| Row | Values |
|-----|--------|
| 0 | [-1.3907, -0.4341, 0.6288, 1.8098, -0.8780] |
| 1 | [-0.4341, 0.6288, 1.8098, 3.1220, 4.5800] |
| 2 | [0.6288, 1.8098, -0.8780, 4.5800, 6.2000] |
| 3 | [-2.1902, 3.1220, 4.5800, 6.2000, 8.0000] |
| 4 | [3.1220, 4.5800, 6.2000, 8.0000, 10.0000] |

3.3 Optimal Greedy Policy π*

| Row | Policy |
|-----|--------|
| 0 | ['R', 'R', 'R', 'D', 'D'] |
| 1 | ['R', 'R', 'R', 'R', 'D'] |
| 2 | ['R', 'D', 'R', 'R', 'D'] |
| 3 | ['R', 'R', 'R', 'R', 'D'] |
| 4 | ['R', 'R', 'R', 'R', 'G'] |

4. In-place vs Standard Value Iteration

| Aspect | Standard | In-place |
|--------|----------|----------|
| Iterations | 10 | 10 |
| Time | 0.0075 s | 0.0040 s |
| Memory | 2 arrays (old/new) | 1 array |
| Convergence | Slightly slower in general | Slightly faster in practice for small grids |
| Complexity | O(|S|·|A|) per iteration | O(|S|·|A|) per iteration |

Notes:

- Both methods produce identical value functions and policies.
- In-place value iteration is slightly faster in this example due to immediate updates.
- Memory efficiency becomes more important for large state spaces; here the difference is negligible.



**Problem 4 — Off-policy Monte Carlo with Importance Sampling**

**1.  Environment and Policies**

•	Behavior policy b(a|s): uniform random (each action probability = 0.25).

•	Target policy π(a|s): deterministic greedy policy from value iteration.

•	Discount factor: γ = 0.9

•	Episodes run: 20,000
Key point: Deterministic target + uniform behavior leads to high variance and sparse updates, making convergence slow.


In [4]:

##2.  Python Implementation 

import numpy as np
import time

ROWS, COLS = 5, 5
ACTIONS = ['R','D','L','U']
ACTION_TO_DELTA = {'R': (0,1), 'D': (1,0), 'L': (0,-1), 'U': (-1,0)}
GAMMA = 0.9
GOAL = (4,4)
REWARDS = np.full((ROWS, COLS), -1.0)
for (r,c) in [(2,2),(3,0),(0,4)]: REWARDS[r,c] = -5.0
REWARDS[GOAL] = 10.0

def step(rc, action):
    r,c = rc
    dr,dc = ACTION_TO_DELTA[action]
    nr, nc = r + dr, c + dc
    return (nr,nc) if 0<=nr<ROWS and 0<=nc<COLS else (r,c)

# Value Iteration to extract target policy
def value_iteration(rewards, gamma=GAMMA, theta=1e-5):
    V = np.zeros_like(rewards)
    while True:
        delta = 0.0
        V_new = V.copy()
        for r in range(ROWS):
            for c in range(COLS):
                if (r,c) == GOAL:
                    V_new[r,c] = rewards[r,c]
                    continue
                q_vals = [rewards[r,c] + gamma * V[step((r,c),a)] for a in ACTIONS]
                V_new[r,c] = max(q_vals)
                delta = max(delta, abs(V_new[r,c]-V[r,c]))
        V = V_new
        if delta < theta:
            break
    policy = np.full((ROWS,COLS), '', dtype=object)
    for r in range(ROWS):
        for c in range(COLS):
            if (r,c)==GOAL: policy[r,c]='G'
            else: policy[r,c]=max(ACTIONS, key=lambda a: rewards[r,c]+gamma*V[step((r,c),a)])
    return V, policy

# Off-policy MC with weighted importance sampling
def off_policy_mc_IS(num_episodes, gamma=GAMMA):
    V = np.zeros((ROWS,COLS))
    C = np.zeros((ROWS,COLS))
    _, target_policy = value_iteration(REWARDS, gamma)
    for ep in range(num_episodes):
        state = (0,0)
        episode = []
        while state != GOAL:
            a = np.random.choice(ACTIONS)
            next_state = step(state, a)
            r = REWARDS[state]
            episode.append((state, a, r))
            state = next_state
        G = 0
        W = 1.0
        for state, a, r in reversed(episode):
            G = gamma*G + r
            r_target = target_policy[state]
            if a != r_target:
                W = 0
                break
            C[state] += W
            V[state] += (W/C[state]) * (G - V[state])
            W *= 1.0
    return V

# Run MC and print results
start_time = time.time()
num_episodes = 20000
V_mc = off_policy_mc_IS(num_episodes)
end_time = time.time()

print("Off-policy MC Estimated Value Function (V):")
for row in V_mc:
    print(np.round(row,4))

V_vi, _ = value_iteration(REWARDS)
mse = np.mean((V_mc - V_vi)**2)
print("\nMean Squared Error vs V* (Value Iteration):", round(mse,5))
print("Execution Time: {:.4f} seconds".format(end_time - start_time))


Off-policy MC Estimated Value Function (V):
[-5.6953 -5.217  -4.6856 -4.0951 -7.439 ]
[-5.217  -4.6856 -4.0951 -3.439  -2.71  ]
[-4.6856 -4.0951 -7.439  -2.71   -1.9   ]
[-8.0951 -3.439  -2.71   -1.9    -1.    ]
[-3.439 -2.71  -1.9   -1.     0.   ]

Mean Squared Error vs V* (Value Iteration): 47.00496
Execution Time: 46.6150 seconds


## 3. Results (20,000 Episodes)
Estimated Value Function V_MC:
Row	Values
0	[-5.6953, -5.2170, -4.6856, -4.0951, -7.4390]
1	[-5.2170, -4.6856, -4.0951, -3.4390, -2.7100]
2	[-4.6856, -4.0951, -7.4390, -2.7100, -1.9000]
3	[-8.0951, -3.4390, -2.7100, -1.9000, -1.0000]
4	[-3.4390, -2.7100, -1.9000, -1.0000, 0.0000]

Mean Squared Error vs V*:  47.00496
Execution Time: 111.4905 seconds

## 4. Discussion
-Most states have low or negative values due to **zero-weight episodes**.  
- Deterministic target + uniform behavior is highly inefficient.  
- To improve: use ε-soft target, per-decision IS, or TD/actor-critic methods.  
- Execution time is high because many episodes are generated but few contribute to updates.